In [1]:
import numpy as np

In [2]:
# env with rewards
env = np.array([
    [-1, -5, -3],
    [0, 1, 2],
    [-1, -10, 10]
])

actions = [1, 2, 3, 4]
n_actions = len(actions)
index_to_action = {
    1: 'LEFT',
    2: 'RIGHT',
    3: 'UP',
    4: 'DOWN'
}

# Q-Action value array with shape (states, actions)
QA = np.random.uniform(0, 0.1, size=(env.size, n_actions))

QA

array([[0.02525094, 0.04528232, 0.09610647, 0.01674091],
       [0.04631241, 0.07643725, 0.02790487, 0.06624687],
       [0.04098774, 0.04763261, 0.01510593, 0.07993323],
       [0.02935304, 0.06887249, 0.04884293, 0.06766028],
       [0.0326565 , 0.03185554, 0.02269977, 0.03870416],
       [0.0458033 , 0.05488222, 0.07618157, 0.03863312],
       [0.07931658, 0.07940986, 0.02716863, 0.07549088],
       [0.04636471, 0.08668528, 0.05362158, 0.01205883],
       [0.08547938, 0.00644371, 0.01001093, 0.09817685]])

In [3]:
# left, right, up, down
def get_available_actions_by_state(state_index):
    """Returns binary mask for available actions: [left, right, up, down]"""
    actions_map = {
        0: [0, 1, 0, 1],  # top-left: right, down
        1: [1, 1, 0, 1],  # top-center: left, right, down
        2: [1, 0, 0, 1],  # top-right: left, down
        3: [0, 1, 1, 1],  # middle-left: right, up, down
        4: [1, 1, 1, 1],  # middle-center: all directions
        5: [1, 0, 1, 1],  # middle-right: left, up, down
        6: [0, 1, 1, 0],  # bottom-left: right, up
        7: [1, 1, 1, 0],  # bottom-center: left, right, up
        8: [1, 0, 1, 0],  # bottom-right: left, up
    }
    return np.array(actions_map[state_index])

def state_matrix_to_index(state):
    return state[0] * 3 + state[1]

def index_to_state_matrix(index):
    return [index // 3, index % 3]

def traverse_state_matrix(state, action):
    """Move in the environment based on action"""
    new_state = state.copy()
    if action == 1:    # left
        new_state[1] -= 1
    elif action == 2:  # right
        new_state[1] += 1
    elif action == 3:  # up
        new_state[0] -= 1
    elif action == 4:  # down
        new_state[0] += 1
    return new_state

def epsilon_greedy_action(state_index, epsilon=0.1):
    """Select action using epsilon-greedy strategy"""
    avail_actions = get_available_actions_by_state(state_index)
    valid_actions = [a for i, a in enumerate(actions) if avail_actions[i]]

    if np.random.random() < epsilon:
        # Explore: random action
        return np.random.choice(valid_actions)
    else:
        # Exploit: best action among valid ones
        q_values = QA[state_index, :].copy()
        for i, a in enumerate(actions):
            if not avail_actions[i]:
                q_values[i] = -np.inf  # Mask invalid actions
        best_action_idx = np.argmax(q_values)
        return actions[best_action_idx]

In [4]:
# Zero out Q-values for unavailable actions
for s in range(env.size):
    avail_actions = get_available_actions_by_state(s)
    QA[s, :] = QA[s, :] * avail_actions

QA

array([[0.        , 0.04528232, 0.        , 0.01674091],
       [0.04631241, 0.07643725, 0.        , 0.06624687],
       [0.04098774, 0.        , 0.        , 0.07993323],
       [0.        , 0.06887249, 0.04884293, 0.06766028],
       [0.0326565 , 0.03185554, 0.02269977, 0.03870416],
       [0.0458033 , 0.        , 0.07618157, 0.03863312],
       [0.        , 0.07940986, 0.02716863, 0.        ],
       [0.04636471, 0.08668528, 0.05362158, 0.        ],
       [0.08547938, 0.        , 0.01001093, 0.        ]])

In [5]:
learning_rate = 0.1      # Alpha - how much we update Q-values
discount_factor = 0.9    # Gamma - importance of future rewards
epsilon = 0.1           # Exploration rate
n_episodes = 1000       # Number of training episodes
max_steps = 100         # Maximum steps per episode

# Track training progress
episode_rewards = []
episode_steps = []

def epsilon_greedy_policy(state_index, epsilon):
    """Choose action using epsilon-greedy policy"""
    available_actions = get_available_actions_by_state(state_index)
    available_indices = np.where(available_actions == 1)[0]

    if np.random.random() < epsilon:
        # Explore: choose random available action
        action_idx = np.random.choice(available_indices)
    else:
        # Exploit: choose best available action
        q_values_masked = QA[state_index, :] * available_actions
        # Set unavailable actions to very low value
        q_values_masked[available_actions == 0] = -np.inf
        action_idx = np.argmax(q_values_masked)

    return action_idx + 1  # Convert to action (1-4)

def get_reward(state):
    """Get reward for being in a state"""
    return env[state[0], state[1]]

# Training loop
print("Starting Q-learning training...")
for episode in range(n_episodes):
    # Initialize episode
    state = [0, 0]  # Start at top-left
    state_index = state_matrix_to_index(state)
    total_reward = 0
    steps = 0

    for step in range(max_steps):
        # Choose action using epsilon-greedy policy
        action = epsilon_greedy_policy(state_index, epsilon)

        # Take action and observe next state
        next_state = traverse_state_matrix(state, action)
        next_state_index = state_matrix_to_index(next_state)

        # Get reward
        reward = get_reward(next_state)
        total_reward += reward

        # Q-learning update
        action_idx = action - 1  # Convert to 0-based index

        # Get max Q-value for next state (considering only available actions)
        next_available_actions = get_available_actions_by_state(next_state_index)
        next_q_values = QA[next_state_index, :] * next_available_actions
        next_q_values[next_available_actions == 0] = -np.inf
        max_next_q = np.max(next_q_values) if np.any(next_available_actions) else 0

        # Q-learning update rule: Q(s,a) = Q(s,a) + α[r + γ*max(Q(s',a')) - Q(s,a)]
        QA[state_index, action_idx] += learning_rate * (
            reward + discount_factor * max_next_q - QA[state_index, action_idx]
        )

        # Move to next state
        state = next_state
        state_index = next_state_index
        steps += 1

        # Terminal condition: reached high reward state or max steps
        if reward == 10 or steps >= max_steps:
            break

    # Track progress
    episode_rewards.append(total_reward)
    episode_steps.append(steps)

    # Decay exploration rate
    epsilon = max(0.01, epsilon * 0.995)

    # Print progress every 100 episodes
    if (episode + 1) % 100 == 0:
        avg_reward = np.mean(episode_rewards[-100:])
        avg_steps = np.mean(episode_steps[-100:])
        print(f"Episode {episode + 1}: Avg Reward = {avg_reward:.2f}, Avg Steps = {avg_steps:.2f}")

print("\nTraining completed!")

Starting Q-learning training...
Episode 100: Avg Reward = 14.43, Avg Steps = 14.18
Episode 200: Avg Reward = 12.89, Avg Steps = 4.20
Episode 300: Avg Reward = 12.71, Avg Steps = 4.10
Episode 400: Avg Reward = 12.79, Avg Steps = 4.08
Episode 500: Avg Reward = 13.04, Avg Steps = 4.08
Episode 600: Avg Reward = 12.92, Avg Steps = 4.04
Episode 700: Avg Reward = 12.98, Avg Steps = 4.04
Episode 800: Avg Reward = 13.01, Avg Steps = 4.06
Episode 900: Avg Reward = 12.98, Avg Steps = 4.04
Episode 1000: Avg Reward = 12.87, Avg Steps = 4.04

Training completed!


In [ ]:
QA

array([[0.00000000e+00, 2.32564103e+01, 0.00000000e+00, 2.82564103e+01],
       [2.58435897e+01, 2.57435897e+01, 0.00000000e+00, 2.97435897e+01],
       [2.32564103e+01, 0.00000000e+00, 0.00000000e+00, 3.02564103e+01],
       [0.00000000e+00, 2.97435897e+01, 2.58435897e+01, 2.58435897e+01],
       [2.82564103e+01, 3.02564103e+01, 2.32564103e+01, 1.82564103e+01],
       [2.97435897e+01, 0.00000000e+00, 2.57435897e+01, 1.00527178e+01],
       [0.00000000e+00, 1.82564096e+01, 2.82564103e+01, 0.00000000e+00],
       [2.58435897e+01, 1.00527178e+01, 2.97435897e+01, 0.00000000e+00],
       [1.02101399e-03, 0.00000000e+00, 5.54924660e-02, 0.00000000e+00]])

In [6]:
for r in range(QA.shape[0]):
  print(f"state {r} best action {index_to_action[QA[r, :].argmax() + 1]}")

state 0 best action DOWN
state 1 best action DOWN
state 2 best action DOWN
state 3 best action RIGHT
state 4 best action RIGHT
state 5 best action DOWN
state 6 best action UP
state 7 best action RIGHT
state 8 best action LEFT
